In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os

gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
tf.config.experimental.set_memory_growth(gpus[1], True)

AUTOTUNE = tf.data.AUTOTUNE

from shared_funcs import multi_label_binary_encode_tensor, multi_label_binary_decode_tensor, get_waveform, split_into_windows, split_into_sequences 

2024-12-07 13:09:00.874809: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-07 13:09:08.275626: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-07 13:09:12.384025: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-07 13:09:36.268127: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-07 13:11:57.845934: W tensorflow/compiler/tf2

In [2]:
ALL_LABELS = tf.constant(['Rctrl', 'p', 'esc', 'g', 'slash', 'down', '7', 'equal', 'w', 'a', 'dash', 'caps', 'l', 'd', 'backspace', 'bracketclose', 'z', '1', 'end', 'Rshift', 'comma', 'c', 'tab', 'b', 'j', 'right', 'Lctrl', 'n', 't', 'f', 'm', 'o', 'apostrophe', 'y', '8', 'space', 'backslash', 's', '9', 'i', 'r', 'bracketopen', 'semicolon', 'q', '5', 'k', '3', 'x', '4', '6', '2', 'Lshift', 'left', 'backtick', 'enter', 'fullstop', 'e', '0', 'h', 'v', 'up', 'u', 'delete'], dtype=tf.string)


I0000 00:00:1733602630.004670 2647090 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733602630.008548 2647090 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733602630.010005 2647090 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733602630.322563 2647090 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

### Load Datasets

In [3]:
# Load MKA Dataset
# mka_path = os.path.dirname(os.getcwd()) + "/data-manipulation/mka_dataset"
# mka_dataset = tf.data.Dataset.load(mka_path)

# Load supplementary dataset
supp_path = os.path.dirname(os.getcwd()) + "/data-manipulation/supp_dataset"
supp_dataset = tf.data.Dataset.load(supp_path)

supp_dataset = supp_dataset.shuffle(128)

# mka_dataset.cache()
supp_dataset.cache()
# mka_dataset = mka_dataset.prefetch(tf.data.AUTOTUNE)
supp_dataset = supp_dataset.prefetch(AUTOTUNE)

# print(f"MKA Dataset {mka_dataset.element_spec}\nLength: {tf.data.experimental.cardinality(mka_dataset).numpy()}")
print(f"Supp Dataset {supp_dataset.element_spec}\nLength: {tf.data.experimental.cardinality(supp_dataset).numpy()}")

spectrogram_dataset = supp_dataset

# Add channel dimension for conv network
spectrogram_dataset = spectrogram_dataset.map(
    lambda spectrogram, label: (tf.expand_dims(spectrogram, axis=-1), label),
    num_parallel_calls=tf.data.AUTOTUNE
)

Supp Dataset (TensorSpec(shape=(78, 552), dtype=tf.float32, name=None), TensorSpec(shape=(78, 63), dtype=tf.int32, name=None))
Length: 752


### Prepare Dataset for Training

In [4]:
dataset_size = sum(1 for _ in spectrogram_dataset)
train_size = int(0.8 * dataset_size)
val_size = int(0.1 * dataset_size)

# Split into train val and test datasets
train_dataset = spectrogram_dataset.take(train_size)
remaining_dataset = spectrogram_dataset.skip(train_size)
val_dataset = remaining_dataset.take(val_size)
test_dataset = remaining_dataset.skip(val_size)

2024-12-07 13:18:20.725253: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Calculate Class Weights
Since the dataset may be imbalanced with different classes, especially with the no key pressed class, we must calculate the class weights so the model is not biased towards the majority class

In [5]:
def calculate_class_weights(labels):
    print(labels.shape)
    num_samples, num_classes = labels.shape
    class_counts = np.sum(labels, axis=0)
    total_counts = np.sum(class_counts)
    
    class_weights = {i: total_counts / (num_classes * class_counts[i]) for i in range(num_classes)}
    return class_weights

# Example usage
all_labels = np.concatenate([label.numpy() for _, label in train_dataset])  # Gather all labels
class_weights = calculate_class_weights(all_labels)
print("Class Weights:", class_weights)

2024-12-07 13:18:20.966767: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


(46878, 63)
Class Weights: {0: 4.242581090407177, 1: 1.4245162785308771, 2: 2.7487145092778897, 3: 1.0492404847243557, 4: 1.711918685602896, 5: inf, 6: 9.293272864701436, 7: 16.263227513227513, 8: 0.9566604419545596, 9: 0.27067785042819714, 10: 2.9128168680407485, 11: 4.065806878306878, 12: 0.5986464115298471, 13: 0.5303226363008972, 14: 0.6945150539456589, 15: 10.84215167548501, 16: 2.7487145092778897, 17: 3.4849773242630384, 18: inf, 19: 2.956950456950457, 20: 2.439484126984127, 21: 0.7228101116990006, 22: 48.78968253968254, 23: 1.3367036312241791, 24: 10.84215167548501, 25: inf, 26: 4.336860670194003, 27: 0.3926735013254128, 28: 0.24672405835490538, 29: 1.4040196414297135, 30: 0.8131613756613757, 31: 0.4040553419435407, 32: 4.4354256854256855, 33: 0.8375911165610737, 34: 7.506105006105006, 35: 0.11595884144903752, 36: 2.439484126984127, 37: 0.3225764134855044, 38: 7.228101116990006, 39: 0.3341759078060448, 40: 0.378948990599476, 41: 12.197420634920634, 42: 3.252645502645503, 43: 4.8

/tmp/ipykernel_2647090/1588371172.py:7: RuntimeWarning: divide by zero encountered in scalar divide
  class_weights = {i: total_counts / (num_classes * class_counts[i]) for i in range(num_classes)}


### Batch and Prefetch datasets

In [6]:
# Batch and prefetch
train_dataset = train_dataset.batch(32).prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.batch(32).prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(buffer_size=AUTOTUNE)

# Model

In [7]:
print(spectrogram_dataset.element_spec)
model = keras.Sequential([
    layers.Input(shape=(78, 552, 1)),
    layers.Conv2D(32, kernel_size=3, activation='relu', padding='same'),
    layers.Conv2D(128, kernel_size=3, activation='relu', padding='same'),
    # layers.Conv2D(256, kernel_size=3, activation='relu', padding='same'),
    layers.MaxPooling2D(pool_size=(1, 2)),
    layers.Dropout(0.3),

    layers.Reshape((78, -1)),
    layers.TimeDistributed(layers.Dense(128, activation='relu')),
    layers.TimeDistributed(layers.Dense(63, activation="sigmoid"))
])

model.summary()

(TensorSpec(shape=(78, 552, 1), dtype=tf.float32, name=None), TensorSpec(shape=(78, 63), dtype=tf.int32, name=None))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 78, 552, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 78, 552, 128)   │        36,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 78, 276, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 78, 276, 128)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 78, 35328)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 78, 128)        │     4,522,112 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 78, 63)         │         8,127 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,567,551 (17.42 MB)

 Trainable params: 4,567,551 (17.42 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=["accuracy", keras.metrics.Precision()] # , 
)

model.fit(
    train_dataset,
    epochs=15,  # Adjust as needed
    validation_data=val_dataset,
    class_weight=class_weights
)

Epoch 1/15


I0000 00:00:1733602758.407815 2648779 service.cc:146] XLA service 0x7fc148004d40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733602758.407841 2648779 service.cc:154]   StreamExecutor device (0): NVIDIA A100 80GB PCIe, Compute Capability 8.0
2024-12-07 13:19:27.940519: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-07 13:19:35.484178: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-12-07 13:19:55.219835: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng33{k2=2,k6=0,k13=2,k14=0,k22=2} for conv (f32[32,32,78,552]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,1,78,552]{3,2,1,0}, f32[32,1,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"ope

 5/19 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.0071 - loss: 1.7306 - precision: 0.0045  

I0000 00:00:1733602882.598865 2648779 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


19/19 ━━━━━━━━━━━━━━━━━━━━ 189s 3s/step - accuracy: 0.0086 - loss: 0.8484 - precision: 0.0046 - val_accuracy: 0.0068 - val_loss: 0.0528 - val_precision: 0.0000e+00
Epoch 2/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.0042 - loss: 0.0581 - precision: 0.0000e+00 - val_accuracy: 8.5470e-04 - val_loss: 0.0565 - val_precision: 0.0000e+00
Epoch 3/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.0031 - loss: 0.0576 - precision: 0.0000e+00 - val_accuracy: 0.0000e+00 - val_loss: 0.0459 - val_precision: 0.0000e+00
Epoch 4/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.0018 - loss: 0.0578 - precision: 0.0000e+00 - val_accuracy: 0.0046 - val_loss: 0.0351 - val_precision: 0.0000e+00
Epoch 5/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.0032 - loss: 0.0539 - precision: 0.0000e+00 - val_accuracy: 0.0053 - val_loss: 0.0400 - val_precision: 0.0000e+00
Epoch 6/15
19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.0265 - loss: 0.0520 - precision: 0.0000e+00 - v

## Evaluate Model

In [9]:
loss, accuracy, precision = model.evaluate(test_dataset)
print(f"Loss: {loss}, Accuracy: {accuracy}, Precision: {precision}")


for spectrogram, label in test_dataset.take(5):
    predictions = model.predict(spectrogram)
    # print(predictions)
    threshold = 0.5
    binary_predictions = (predictions > threshold).astype(int)

    # Output predictions
    # print("Raw Predictions:", predictions)
    binary_predictions_tensor = tf.convert_to_tensor(binary_predictions, dtype=tf.int32)
    print("Binary Predictions:", multi_label_binary_decode_tensor(tf, binary_predictions_tensor))

3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 2s/step - accuracy: 0.0073 - loss: 0.0300 - precision: 0.0000e+00  
Loss: 0.030150124803185463, Accuracy: 0.007085021585226059, Precision: 0.0
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

## Optimize Model